<a href="https://colab.research.google.com/github/omor-niloy/Parallel-Processing-Lab/blob/main/PP_phone_book_search2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile phonebook_search.cu
#include <bits/stdc++.h>
using namespace std;
#include <cuda.h>

struct Contact{
    char name[65];
    char phone_number[65];
};

void read_phonebook(string file_names, vector<Contact> &phoneBook)
{
    int cnt = 10;

        ifstream f(file_names);
        string line;
        while (getline(f, line)) {
            if (line.empty()) continue;
            int comma = line.find(",");
            if (comma == string::npos) continue;
            // contacts.push_back({line.substr(1, comma - 2), line.substr(comma + 2, line.size() - comma - 3)});

            string name = (line.substr(1, comma - 2));
            string phoneNum = (line.substr(comma + 2, line.size() - comma - 3));
            Contact c;
            strcpy(c.name, name.c_str());
            strcpy(c.phone_number, phoneNum.c_str());
            phoneBook.push_back(c);
            //cnt--;
            if(cnt < 0)break;
        }

}

__device__ bool check(char* str1, char* str2){
    for(int i = 0; str1[i] != '\0'; i++){
        int flag = 1;
        for(int j = 0; str2[j] != '\0' ; j++){
            if(str1[i + j] != str2[j]){
                flag = 0;
                break;
            }
        }
        if(flag == 1) return true;
    }
    return false;
}


__global__ void myKernel(Contact* phoneBook, char* pat, int offset){
    int threadNumber = threadIdx.x + offset;
    if(check(phoneBook[threadNumber].name, pat)){
        printf("%s %s\n", phoneBook[threadNumber].name, phoneBook[threadNumber].phone_number);
    }
}

int main(int argc, char* argv[])
{
    int threadLimit = atoi(argv[2]);

    vector<Contact> phoneBook;
    read_phonebook( "/content/drive/MyDrive/Copy of labtest_dataset1.txt" , phoneBook);


    string search_name = argv[1];
    char pat[65];
    strcpy(pat, search_name.c_str());


    char* d_pat;
    cudaMalloc(&d_pat, 65); //memory allocation
    cudaMemcpy(d_pat, pat, 65, cudaMemcpyHostToDevice); //copying to device

    int n = phoneBook.size();
    Contact* d_phoneBook;
    cudaMalloc(&d_phoneBook, n*sizeof(Contact));
    cudaMemcpy(d_phoneBook, phoneBook.data(), n * sizeof(Contact), cudaMemcpyHostToDevice);


    int bakiAche = n;
    int offset = 0;
    while(bakiAche > 0){
        int batchSize = min(threadLimit, bakiAche);
        myKernel<<<1,batchSize>>>(d_phoneBook, d_pat, offset);
        cudaDeviceSynchronize();

        bakiAche -= batchSize;
        offset += batchSize;
    }
}

Overwriting phonebook_search.cu


In [ ]:
!nvcc -arch=sm_75 phonebook_search.cu -o phonebook_search

In [ ]:
!time ./phonebook_search NOURIN 1 > output.txt


real	3m7.455s
user	3m2.190s
sys	0m3.963s


In [ ]:
!time ./phonebook_search SAKIA 2 > output.txt


real	1m43.024s
user	1m38.235s
sys	0m3.942s


In [ ]:
!time ./phonebook_search SAKIA 4 > output.txt


real	1m3.002s
user	0m58.459s
sys	0m3.789s


In [ ]:
!time ./phonebook_search NOURIN 500 > output.txt


real	0m9.947s
user	0m5.928s
sys	0m3.762s
